In [9]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Any
import uuid


class QdrantConnector:
      
    def __init__(
        self, 
        qdrant_host: str = "localhost", 
        qdrant_port: int = 6333,
        embedding_model: str = "jinaai/jina-embeddings-v2-small-en",
        collection_name: str = "pdf_documents"
    ):
        self.qdrant_client = QdrantClient(host=qdrant_host, port=qdrant_port)
        self.embedding_model = SentenceTransformer(embedding_model)
        self.collection_name = collection_name
        self._ensure_collection_exists()

    @property
    def sentence_splitter(self):
        return RecursiveCharacterTextSplitter(
            chunk_size=500,     # Number of characters per chunk
            chunk_overlap=50,   # Overlap to maintain context
            separators=["\n\n", "\n", ".", " "]  # Prioritize splitting on paragraph, newline, and sentence boundaries
        )
    
    def _ensure_collection_exists(self):
        """Check and create collection in Qdrant if it doesn't exist"""
        try:
            collections = self.qdrant_client.get_collections()
            collection_names = [col.name for col in collections.collections]
            
            if self.collection_name not in collection_names:
                # Create collection with appropriate parameters
                vector_size = self.embedding_model.get_sentence_embedding_dimension()
                print(f"Creating collection '{self.collection_name}' with vector size {vector_size}")
                
                self.qdrant_client.create_collection(
                    collection_name=self.collection_name,
                    vectors_config=VectorParams(
                        size=vector_size,
                        distance=Distance.COSINE
                    )
                )
                print(f"Created collection '{self.collection_name}'")
            else:
                print(f"Collection '{self.collection_name}' already exists")
        except Exception as e:
            print(f"Error creating collection: {e}")

    def upload_to_qdrant(
        self, 
        text: str,
        metadata: dict[str, Any] = None
    ) -> None:
        """
        Upload chunks and embeddings to Qdrant
        
        Args:
            chunks: List of text chunks
            embeddings: Embeddings for chunks
            metadata: Additional metadata (e.g., filename, page)
        
        Returns:
            List of IDs of added points
        """
        sentence_chunks = self.sentence_splitter.split_text(text)
        print(f"text spliited into {len(sentence_chunks)} chunks")
        for i, chunk in enumerate(sentence_chunks):
            point_id = str(uuid.uuid4())
            embedding = self.embedding_model.encode(chunk)

            payload = {
                "text": chunk,
                "chunk_index": i,
                "chunk_length": len(chunk)
            }
            if metadata:
                payload.update(metadata)

            point = PointStruct(
                id=point_id,
                vector=embedding.tolist(),
                payload=payload
            )
            try:
                self.qdrant_client.upsert(
                    collection_name=self.collection_name,
                    points=[point]
                )
                print(f"Uploaded point {point_id} to Qdrant")
            except Exception as e:
                print(f"Error uploading to Qdrant: {e}")
                return None

    def search_similar(self, query: str, limit: int = 5) -> list[dict]:
            """
            Search for similar chunks to query
            
            Args:
                query: Text query
                limit: Maximum number of results
            
            Returns:
                List of similar chunks with metadata
            """
            # Create embedding for query
            query_embedding = self.embedding_model.encode([query])[0]
            
            # Search in Qdrant
            search_results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_embedding.tolist(),
                limit=limit
            )
            
            results = []
            for result in search_results:
                results.append({
                    "text": result.payload["text"],
                    "score": result.score,
                    "file_name": result.payload.get("file_name", "unknown"),
                    "chunk_index": result.payload.get("chunk_index", 0)
                })
            
            return results


### Semantic search - model comparison

In [10]:
qdrant_connector = QdrantConnector(
    embedding_model = "jinaai/jina-embeddings-v2-small-en",
    collection_name = "pdf_documents"
)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Collection 'pdf_documents' already exists


In [13]:
results = qdrant_connector.search_similar("według jakich zasad odbywa się procedura zgłaszania i wydawania tematów prac dyplomowych przez nauczycieli akademickich dla studentów poszczególnych kierunków ?", limit=5)

/var/folders/k2/8_xbq0kj3yq1ydl1c3mcpp580000gn/T/ipykernel_43622/1337551721.py:115: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = self.qdrant_client.search(


In [14]:
results

[{'text': 'oraz umiejętnoś ci prowadzenia badań naukowych . 115 54,8% \nZajęciom z obszarów nauk humanistycznych lub nauk społecznych \n(w przypadku kierunków studiów przypisanych do obszarów innych \nniż odpowiednio nauki humanistyczne lub nau ki społeczne) . 10  \nPrzedmiotom  obieralnym  (zajęciom  do wyboru ). 63 30% \nPraktykom zawodowym (jeżeli progra m studiów przewiduje praktyki) . 6  \nZ wykorzystaniem metod i technik kształcenia na odległość.  3 1,4%',
  'score': 0.66370624,
  'file_name': '4_Program_studiow_zarządzanie i inżynieria produkcji_WIM_I_stacj_niest_og.pdf',
  'chunk_index': 3},
 {'text': 'technik druku 3D.  \n \n \n4. Opis kompetencji oczekiwanych od kandydata ubiegającego się o przyjęcie na studia  \n \nNa studia I stopnia może być przyjęta osoba, która posiada świadectwo dojrzałości lub inny \ndokument, o którym mowa w art. 69 ust. 2 ustawy Prawo o szkolnictwie wyższym i nauce. Ponadto, od \nkandydatów na studia I stopnia na kierunku zarządzanie i inżynieria pro

In [10]:
qdrant_connector_jinaai = QdrantConnector(
    embedding_model = "jinaai/jina-embeddings-v2-small-en",
    collection_name = "pdf_documents_jinaai"
)

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Created collection 'pdf_documents_jinaai'


In [12]:
import os
import PyPDF2


class PDFToQdrant:
    def __init__(self, qdrant_connector: QdrantConnector):
        """
        Initialize PDF to Qdrant processing system
        
        Args:
            qdrant_host: Qdrant server host address
            qdrant_port: Qdrant server port
            embedding_model: Model for creating embeddings
            collection_name: Collection name in Qdrant
        """
        self.qdrant_connector = qdrant_connector
    
    def _extract_pdf(self, pdf_path: str) -> str:
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text()
            return text
        except Exception as e:
            print(f"Error reading PDF: {e}")
            return ""
    
    def process_pdf(
        self, 
        pdf_path: str, 
        chunk_size: int = 1000, 
        overlap: int = 100
    ) -> None:
        print(f"Processing file: {pdf_path}")
        text = self._extract_pdf(pdf_path)
        if not text.strip():
            return {"error": "Failed to extract text from PDF"}
        
        print(f"Extracted {len(text)} characters of text")

        metadata = {
            "file_name": os.path.basename(pdf_path),
            "file_path": pdf_path,
            "chunk_size": chunk_size,
            "overlap": overlap
        }
        
        self.qdrant_connector.upload_to_qdrant(text, metadata)

In [25]:
from pathlib import Path
import os

DATA_PATH = os.path.dirname(os.getcwd()) + "/data"

pdf_processor = PDFToQdrant(qdrant_connector=qdrant_connector_jinaai)
for file in [f for f in os.listdir(DATA_PATH) if f.endswith('.pdf')]:
    pdf_processor.process_pdf(f"{DATA_PATH}/{file}", chunk_size=500, overlap=100)

Processing file: /Users/tmatuszewski/Projects/pdf-research-assistant/data/4_Program_studiow_zarządzanie i inżynieria produkcji_WIM_I_stacj_niest_og.pdf


BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Extracted 89372 characters of text
text spliited into 198 chunks
Uploaded point adc93b6e-1903-4360-a5d2-b0a7308cabf9 to Qdrant
Uploaded point ae47f5aa-5a9e-4fd0-ad21-c0d273e67b93 to Qdrant
Uploaded point f8033b1a-74f4-45de-b0b7-5a92d57d8468 to Qdrant
Uploaded point 30a0c82d-296a-4275-9254-bf7db7748151 to Qdrant
Uploaded point 4660173e-8ca7-4f45-a8f5-0f9cae665e1a to Qdrant
Uploaded point b259a79d-2fd6-4d14-bf3e-47a28eab1c57 to Qdrant
Uploaded point 6bda562e-d64e-4db8-8841-a96d40b94928 to Qdrant
Uploaded point bbc97136-8acb-4ef1-a56f-ecd0d2f2c184 to Qdrant
Uploaded point a37245de-6083-4748-abc5-24cc840a260d to Qdrant
Uploaded point 4686b32f-d8fb-4eb9-8e1c-b2945f177e12 to Qdrant
Uploaded point 89df9a61-e9b5-4913-be9d-ad2f00d266f7 to Qdrant
Uploaded point d377e343-398e-493e-9963-ee7412dd5ffa to Qdrant
Uploaded point b3c2bfb4-a75c-4758-836b-971403614d30 to Qdrant
Uploaded point f542e579-c8a5-4c4f-9db2-975d19e6a907 to Qdrant
Uploaded point 14d78f70-99f4-43ef-bce9-7bf8a17dadf1 to Qdrant
Uploa

In [26]:
results_jinaai = qdrant_connector_jinaai.search_similar("jaka uchwała reguluje zasady dotyczące zapewnienia jakości kształcenia na Politechnice Poznańskiej?", limit=5)

/var/folders/k2/8_xbq0kj3yq1ydl1c3mcpp580000gn/T/ipykernel_53882/1887929910.py:114: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = self.qdrant_client.search(


In [27]:
results_jinaai

[{'text': 'nr 45 Senatu Akademickiego Politechniki Poznańskiej z dnia 31 maja 2021 roku w sprawie Uczelnianego \nSystemu Zapewnienia Jakości Kształcenia. Ponadto, regulacje związane z zapewnieniem jakości  \nkształcenia zawarte są również w Statucie Politechniki Poznańskiej oraz Regulaminie studiów pierwszego \ni drugiego stopnia (Uchwała Nr 42/2020 -2024 Senatu Akademickiego Politechniki Poznańskiej z dnia',
  'score': 0.8444388,
  'file_name': '4_Program_studiow_zarządzanie i inżynieria produkcji_WIM_I_stacj_niest_og.pdf',
  'chunk_index': 116},
 {'text': 'punktów wymaganych do uzyskania kwalifikacji na poziomie 6 PRK, dla ki erunku zarządzanie i inżynieria \nprodukcji.  \n \n \nII. Informacje uzupełniające  \n \n1. Koncepcja kształcenia  oraz zgodność efektów uczenia się z potrzebami rynku pracy  \n \nMisją Wydziału jest kształcenie wysokokwalifikowanych kadr w obszarze inżynierii mechanicznej,  \nw ścisłym związku z prowadzonymi na Wydziale pracami naukowymi i badawczo -rozwojowymi

In [39]:
from qdrant_client import QdrantClient
from qdrant_client import models
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Any
import uuid


class QdrantConnectorHybrid:
      
    def __init__(
        self, 
        qdrant_host: str = "localhost", 
        qdrant_port: int = 6333,
        embedding_model: str = "jinaai/jina-embeddings-v2-small-en",
        collection_name: str = "pdf_documents"
    ):
        self.qdrant_client = QdrantClient(host=qdrant_host, port=qdrant_port)
        self.embedding_model = embedding_model
        self.collection_name = collection_name
        self._ensure_collection_exists()

    @property
    def sentence_splitter(self):
        return RecursiveCharacterTextSplitter(
            chunk_size=500,     # Number of characters per chunk
            chunk_overlap=50,   # Overlap to maintain context
            separators=["\n\n", "\n", ".", " "]  # Prioritize splitting on paragraph, newline, and sentence boundaries
        )
    
    def _ensure_collection_exists(self):
        """Check and create collection in Qdrant if it doesn't exist"""
        try:
            collections = self.qdrant_client.get_collections()
            collection_names = [col.name for col in collections.collections]
            
            if self.collection_name not in collection_names:
        
                self.qdrant_client.create_collection(
                    collection_name="pdf_documents_sparse_and_dense",
                    vectors_config={
                        # Named dense vector for jinaai/jina-embeddings-v2-small-en
                        "jina-small": models.VectorParams(
                            size=512,
                            distance=models.Distance.COSINE,
                        ),
                    },
                    sparse_vectors_config={
                        "bm25": models.SparseVectorParams(
                            modifier=models.Modifier.IDF,
                        )
                    }
)
                print(f"Created collection '{self.collection_name}'")
            else:
                print(f"Collection '{self.collection_name}' already exists")
        except Exception as e:
            print(f"Error creating collection: {e}")

    def upload_to_qdrant(
        self, 
        text: str,
        metadata: dict[str, Any] = None
    ) -> None:
        """
        Upload chunks and embeddings to Qdrant
        
        Args:
            chunks: List of text chunks
            embeddings: Embeddings for chunks
            metadata: Additional metadata (e.g., filename, page)
        
        Returns:
            List of IDs of added points
        """
        sentence_chunks = self.sentence_splitter.split_text(text)
        print(f"text spliited into {len(sentence_chunks)} chunks")
        for i, chunk in enumerate(sentence_chunks):
            point_id = str(uuid.uuid4())
            payload = {
                "text": chunk,
                "chunk_index": i,
                "chunk_length": len(chunk)
            }
            if metadata:
                payload.update(metadata)
            try:
                self.qdrant_client.upsert(
                    collection_name=self.collection_name,
                    points=[
                        models.PointStruct(
                            id=uuid.uuid4().hex,
                            vector={
                                "jina-small": models.Document(
                                    text=chunk,
                                    model=self.embedding_model,
                                ),
                                "bm25": models.Document(
                                    text=chunk, 
                                    model="Qdrant/bm25",
                                ),
                            },
                            payload=payload
                        )
                    ]
                )
                print(f"Uploaded point {point_id} to Qdrant")
            except Exception as e:
                print(f"Error uploading to Qdrant: {e}")
                return None

    def search_similar(self, query: str, limit: int = 5) -> list[str]:
            results = self.qdrant_client.query_points(
                collection_name=self.collection_name,
                prefetch=[
                    models.Prefetch(
                        query=models.Document(
                            text=query,
                            model=self.embedding_model,
                        ),
                        using="jina-small",
                        limit=(10 * limit),
                    ),
                ],
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25", 
                ),
                using="bm25",
                limit=limit,
                with_payload=True,
            ) 
            return [result.payload["text"] for result in  results.points] 


In [40]:
hybrid_qdrant_connector = QdrantConnectorHybrid(
    collection_name = "pdf_documents_sparse_and_dense"
)

Collection 'pdf_documents_sparse_and_dense' already exists


In [35]:
from pathlib import Path
import os

DATA_PATH = os.path.dirname(os.getcwd()) + "/data"

pdf_processor = PDFToQdrant(qdrant_connector=hybrid_qdrant_connector)
for file in [f for f in os.listdir(DATA_PATH) if f.endswith('.pdf')]:
    pdf_processor.process_pdf(f"{DATA_PATH}/{file}", chunk_size=500, overlap=100)

Processing file: /Users/tmatuszewski/Projects/pdf-research-assistant/data/4_Program_studiow_zarządzanie i inżynieria produkcji_WIM_I_stacj_niest_og.pdf
Extracted 89372 characters of text
text spliited into 198 chunks


2025-08-21 11:55:02.825 | ERROR    | fastembed.common.model_management:download_model:430 - Could not download model from HuggingFace: (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=None)"), '(Request ID: 686abcb9-2cad-49aa-81a2-0f0f3944093e)') Falling back to other sources.
2025-08-21 11:55:02.827 | ERROR    | fastembed.common.model_management:download_model:452 - Could not download model from either source, sleeping for 3.0 seconds, 2 retries left.
Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 30.01it/s]


Uploaded point 28bf8867-9f4a-4c06-9b44-2840afc1d28f to Qdrant
Uploaded point 697227e8-8af2-4c1e-a0c6-e5ec1aa9087c to Qdrant
Uploaded point 0979f7c6-e962-4b01-9225-f46dbb344e95 to Qdrant
Uploaded point 09347d10-1e70-4834-8d67-7754e102f296 to Qdrant
Uploaded point e75d298b-20c9-4172-85cb-4c9aa1513bf1 to Qdrant
Uploaded point 1444670b-f6f7-402c-941a-d6fe567fe8bc to Qdrant
Uploaded point c285ad2e-dd3c-40c5-88d2-e3380dab18c3 to Qdrant
Uploaded point 647a1eae-d89e-488a-a3ba-e11eef25ddf5 to Qdrant
Uploaded point f7d9812f-8baf-4655-934c-116f402084fd to Qdrant
Uploaded point 672a7df5-0fb7-4a06-bed5-3367f4d96e61 to Qdrant
Uploaded point 1d88a4b7-ac1e-4b8c-8a1f-a321f76f9f5d to Qdrant
Uploaded point a7729d31-74e5-4b1b-9c7d-d00ef3b704ad to Qdrant
Uploaded point 6d9d8ce7-0388-4269-b81c-13c04890c49c to Qdrant
Uploaded point 0fe6e53d-be08-470e-9514-a970225c4381 to Qdrant
Uploaded point 1f344f88-abb9-45de-b5fe-9ad38d8e4156 to Qdrant
Uploaded point d27e6de0-1e7a-4791-84a8-1ef81538296c to Qdrant
Uploaded

In [42]:
query = "jaka uchwała reguluje zasady dotyczące zapewnienia jakości kształcenia na Politechnice Poznańskiej?"
results_hybrid = hybrid_qdrant_connector.search_similar(query=query, limit=5)
for result in results_hybrid:
    print(result)

absolwentów kierunku zarządzanie i inżynieria produkcji , można stwierdzić, że znajdują zatrudnienie  
w podobnym okresie czasu jak absolwenci innych kierunków inżynieryjno -technicznych a wskaźnik 
bezrobocia był czterokrotnie niższy . 
 
 
2. Opis działań na rzecz doskonalenia programu studiów oraz zapewniania jakości 
kształcenia  
 
Zasady dotyczące zapewnienia jakości kształcenia na Politechnice Poznańskiej regulują Uchwała
nr 45 Senatu Akademickiego Politechniki Poznańskiej z dnia 31 maja 2021 roku w sprawie Uczelnianego 
Systemu Zapewnienia Jakości Kształcenia. Ponadto, regulacje związane z zapewnieniem jakości  
kształcenia zawarte są również w Statucie Politechniki Poznańskiej oraz Regulaminie studiów pierwszego 
i drugiego stopnia (Uchwała Nr 42/2020 -2024 Senatu Akademickiego Politechniki Poznańskiej z dnia
Warunki i tryb przyjmowania na studia ustalane są na dany rok akademicki. Dlatego aktualne zasady 
i harmonogram postępowania rekrutacyjnego należy sprawdzić w Uchwale Se

In [43]:
results_jinaai = qdrant_connector_jinaai.search_similar(query, limit=5)
results_jinaai

/var/folders/k2/8_xbq0kj3yq1ydl1c3mcpp580000gn/T/ipykernel_53882/1887929910.py:114: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = self.qdrant_client.search(


[{'text': 'nr 45 Senatu Akademickiego Politechniki Poznańskiej z dnia 31 maja 2021 roku w sprawie Uczelnianego \nSystemu Zapewnienia Jakości Kształcenia. Ponadto, regulacje związane z zapewnieniem jakości  \nkształcenia zawarte są również w Statucie Politechniki Poznańskiej oraz Regulaminie studiów pierwszego \ni drugiego stopnia (Uchwała Nr 42/2020 -2024 Senatu Akademickiego Politechniki Poznańskiej z dnia',
  'score': 0.8444388,
  'file_name': '4_Program_studiow_zarządzanie i inżynieria produkcji_WIM_I_stacj_niest_og.pdf',
  'chunk_index': 116},
 {'text': 'punktów wymaganych do uzyskania kwalifikacji na poziomie 6 PRK, dla ki erunku zarządzanie i inżynieria \nprodukcji.  \n \n \nII. Informacje uzupełniające  \n \n1. Koncepcja kształcenia  oraz zgodność efektów uczenia się z potrzebami rynku pracy  \n \nMisją Wydziału jest kształcenie wysokokwalifikowanych kadr w obszarze inżynierii mechanicznej,  \nw ścisłym związku z prowadzonymi na Wydziale pracami naukowymi i badawczo -rozwojowymi